In [1]:
#get urls
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re

#collecting all html pages
# from urllib.request import urlopen, Request
# from bs4 import BeautifulSoup
import re
import os

#get tsv files
# from urllib.request import urlopen, Request
# from bs4 import BeautifulSoup
# import re
# import os
import csv
from langdetect import detect
from progress.bar import IncrementalBar

In [14]:
#pre-process all the information
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from unidecode import unidecode
#import re

#Create your index! - vocabularyGenerator
#import csv
import json
from progress.counter import Counter
#import os

from prettytable import PrettyTable
import textwrap

import math
import heapq

In [3]:
import q1
import q2

# 1. Data collection

### 1.1. Get the list of books

In [5]:
#q1.get_urls()

### 1.2. Crawl books

In [6]:
#q1.collect_html_pages()

### 1.3 Parse downloaded pages

In [7]:
#q1.get_all_tsv()

# 2. Search Engine

In [4]:
##pre-process all the information
q2.download()

[nltk_data] Downloading package stopwords to C:\Users\Simone-
[nltk_data]     MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Simone-
[nltk_data]     MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 2.1. Conjunctive query

#### 2.1.1 Create your index!

##### Create vocabulary

In [5]:
vocabulary = dict()

In [48]:
bar = Counter('Progress: ')

for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'
    
    for file in os.scandir(path):
        with open(path + file.name) as tsv_file:
            try:
                tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
                row = tsv_reader.__next__()['plot']
            
                for word in q2.preprocessor(row):
                    if word not in vocabulary.keys():
                        vocabulary[word] = len(vocabulary)
            except:
                #print(f'{path + file.name}')
                continue
        bar.next()
bar.finish()

with open('generated/vocabulary.json', 'w') as json_file:
    json.dump(vocabulary, json_file)

print('File generated.')

File generated.


##### create invertedIndex

In [52]:
invertedIndex = dict()

with open('generated/vocabulary.json', 'r') as json_file:
    vocabulary = json.load(json_file)

for word in vocabulary:
    invertedIndex[vocabulary[word]] = []

bar = Counter('Progress: ')
for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'
    
    for file in os.scandir(path):
        with open(path + file.name) as tsv_file:
            documentId = re.sub(r'(article\_|\.tsv)', '', file.name)
            tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
            try:
                row = tsv_reader.__next__()['plot']

                for word in q2.preprocessor(row):
                    if documentId not in invertedIndex[vocabulary[word]]:
                        invertedIndex[vocabulary[word]].append(documentId)
            except:
                #print(f'{path + file.name}')
                continue
        bar.next()
bar.finish()


with open('generated/invertedIndex.json', 'w') as json_file:
    json.dump(invertedIndex, json_file)

print('File generated.')

File generated.


#### 2.1.2 Execute the query

In [8]:
query = input('Query: ')
query = q2.preprocessor(query)
resultId = q2.search(query)

for i in resultId:
    with open(f'TSV_books/list_page_{i // 100 + 1}/article_{i}.tsv') as tsv_file:
        tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
        row = tsv_reader.__next__()

    with open('generated/url_collected.txt') as urls:
        for j, url in enumerate(urls):
            if j == i:
                print(f"Title: {row['bookTitle']}\nUrl: {url}\nPlot:\n{row['plot']}")
                print('--------------------------------------------------')

Query: survival games
Title: The Hunger Games
Url: https://www.goodreads.com/book/show/2767052-the-hunger-games

Plot:
Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning? In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages Could you survive on your own in the wild, with every one out to make sure you don't live to see the morning? In the ruins of a place once known as North America lies the nation of Panem, a shining Capitol surrounded by twelve outlying districts. The Capitol is harsh and cruel and keeps the districts in line by forcing them all to send one boy and one girl between the ages of twelve and eighteen to participate in the annual Hunger Games, a fight to the death on live TV. Sixteen-year-old Katniss Everd

Title: Skin Game
Url: https://www.goodreads.com/book/show/19486421-skin-game

Plot:
Harry Dresden, Chicago's only professional wizard, is about to have a very bad dayâ€¦ Because as Winter Knight to the Queen of Air and Darkness, Harry never knows what the scheming Mab might want him to do. Usually, itâ€™s something awful. He doesnâ€™t know the half of itâ€¦ Mab has just traded Harryâ€™s skills to pay off one of her debts. And now he must help a group of supernatural v Harry Dresden, Chicago's only professional wizard, is about to have a very bad dayâ€¦ Because as Winter Knight to the Queen of Air and Darkness, Harry never knows what the scheming Mab might want him to do. Usually, itâ€™s something awful. He doesnâ€™t know the half of itâ€¦ Mab has just traded Harryâ€™s skills to pay off one of her debts. And now he must help a group of supernatural villainsâ€”led by one of Harryâ€™s most dreaded and despised enemies, Nicodemus Archleoneâ€”to break into the highest-security vault in town

Title: Glow
Url: https://www.goodreads.com/book/show/7857713-wolfsbane

Plot:
One of School Library Journal â€™s Best Fiction Books of 2011 If a violent battle destroyed the only world youâ€™ve ever known, would you be brave enough to save who was left? Would love be strong enough to survive the fight? Either way, thereâ€™s no turning back. The Empyrean is the only home 15-year-old Waverly has ever known. Part of the first generation to be successfully conce One of School Library Journal â€™s Best Fiction Books of 2011 If a violent battle destroyed the only world youâ€™ve ever known, would you be brave enough to save who was left? Would love be strong enough to survive the fight? Either way, thereâ€™s no turning back. The Empyrean is the only home 15-year-old Waverly has ever known. Part of the first generation to be successfully conceived in deep space, she and her boyfriend Kieran will be pioneers ofÂ New Earth.Â Waverly knows she must marry young in order to have children who can ca

Title: The End of The Computer
Url: https://www.goodreads.com/book/show/23400620-odes

Plot:
Travel to the edge of The End of the world with some of the most interesting and enjoyable people you'll ever meet. Share the spine-chilling thrills and great emotional epiphanies as this company of friends are forced into the adventure of leaving their hollow lives in Moscow and Berkeley and becoming the earth mothers and fathers of Thunder Valley... if any of them survive Travel to the edge of The End of the world with some of the most interesting and enjoyable people you'll ever meet. Share the spine-chilling thrills and great emotional epiphanies as this company of friends are forced into the adventure of leaving their hollow lives in Moscow and Berkeley and becoming the earth mothers and fathers of Thunder Valley... if any of them survive the journey. Thrill with the discovery that a hypercomputer can be fun, kind, happy... and the most dangerous person in existence to those who would dest

Title: Blood Moon
Url: https://www.goodreads.com/book/show/13650334-legacy-of-a-dreamer

Plot:
Guilty until you survive all zonesâ€¦The Orcus Games is ready to put you on trialâ€¦ The lives of a witch and wizard couple, a new vampire and a rogue lycan intertwine in three magical tales of love, loss and revenge. The Craft - When witch and wizard couple, Heather and Phoenix make that fatal mistake on that, â€˜Blood Moonâ€™ night, they never thought they would be forced to worki Guilty until you survive all zonesâ€¦The Orcus Games is ready to put you on trialâ€¦ The lives of a witch and wizard couple, a new vampire and a rogue lycan intertwine in three magical tales of love, loss and revenge. The Craft - When witch and wizard couple, Heather and Phoenix make that fatal mistake on that, â€˜Blood Moonâ€™ night, they never thought they would be forced to working with a loner werewolf butâ€¦is he the only one who saw the, â€˜accidentalâ€™ fatal act? The Clan â€“ Ardan would do anything for hi

### 2.2 Conjunctive query & Ranking score

#### 2.2.1 Inverted index

In [23]:
invertedIndexTfidf = dict()
documentNorm = dict()

with open('generated/vocabulary.json', 'r') as json_file:
    vocabulary = json.load(json_file)

with open('generated/invertedIndex.json', 'r') as json_file:
    invertedIndex = json.load(json_file)

for word in vocabulary:
    invertedIndexTfidf[vocabulary[word]] = []

n = 0
for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'
    n += len([1 for _ in os.scandir(path)])

bar = Counter('Progress: ')
for i in range(1, 301):
    path = f'TSV_books/list_page_{i}/'

    for file in os.scandir(path):
        norm = []
        with open(path + file.name) as tsv_file:
            documentId = re.sub(r'(article\_|\.tsv)', '', file.name)
            tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
            try:
                plot = tsv_reader.__next__()['plot']
            except:
                print(f'{path + file.name}')
                continue
                
            words = q2.preprocessor(plot)
            doneWords = []
            for word in words:
                if word not in doneWords:
                    tf = len([w for w in words if w == word])
                    N = len(invertedIndex[str(vocabulary[word])])
                    idf = math.log(n/N)
                    invertedIndexTfidf[vocabulary[word]].append((int(documentId), tf * idf))
                    doneWords.append(word)

                norm.append(invertedIndexTfidf[vocabulary[word]][-1][1])

                
        documentNorm[documentId] = math.sqrt(sum([x**2 for x in norm]))

        bar.next()
bar.finish()


with open('generated/invertedIndexTfidf.json', 'w') as json_file:
    json.dump(invertedIndexTfidf, json_file)

with open('generated/documentNorm.json', 'w') as json_file:
    json.dump(documentNorm, json_file)

print('File generated.')

TSV_books/list_page_1/article_15.tsv
TSV_books/list_page_1/article_22.tsv
TSV_books/list_page_1/article_26.tsv
TSV_books/list_page_1/article_30.tsv
TSV_books/list_page_1/article_51.tsv
TSV_books/list_page_1/article_56.tsv
TSV_books/list_page_1/article_63.tsv
TSV_books/list_page_1/article_68.tsv
TSV_books/list_page_1/article_69.tsv
TSV_books/list_page_1/article_75.tsv
TSV_books/list_page_1/article_82.tsv
TSV_books/list_page_1/article_83.tsv
TSV_books/list_page_1/article_84.tsv
TSV_books/list_page_2/article_118.tsv
TSV_books/list_page_2/article_124.tsv
TSV_books/list_page_2/article_138.tsv
TSV_books/list_page_2/article_143.tsv
TSV_books/list_page_2/article_160.tsv
TSV_books/list_page_2/article_169.tsv
TSV_books/list_page_2/article_183.tsv
TSV_books/list_page_2/article_189.tsv
TSV_books/list_page_3/article_202.tsv
TSV_books/list_page_3/article_245.tsv
TSV_books/list_page_3/article_266.tsv
TSV_books/list_page_3/article_286.tsv
TSV_books/list_page_3/article_299.tsv
TSV_books/list_page_4/art

TSV_books/list_page_31/article_3013.tsv
TSV_books/list_page_31/article_3025.tsv
TSV_books/list_page_31/article_3032.tsv
TSV_books/list_page_31/article_3045.tsv
TSV_books/list_page_31/article_3057.tsv
TSV_books/list_page_31/article_3074.tsv
TSV_books/list_page_31/article_3076.tsv
TSV_books/list_page_32/article_3100.tsv
TSV_books/list_page_32/article_3105.tsv
TSV_books/list_page_32/article_3109.tsv
TSV_books/list_page_32/article_3123.tsv
TSV_books/list_page_32/article_3131.tsv
TSV_books/list_page_32/article_3150.tsv
TSV_books/list_page_32/article_3152.tsv
TSV_books/list_page_32/article_3153.tsv
TSV_books/list_page_32/article_3173.tsv
TSV_books/list_page_32/article_3192.tsv
TSV_books/list_page_33/article_3206.tsv
TSV_books/list_page_33/article_3209.tsv
TSV_books/list_page_33/article_3212.tsv
TSV_books/list_page_33/article_3219.tsv
TSV_books/list_page_33/article_3248.tsv
TSV_books/list_page_33/article_3256.tsv
TSV_books/list_page_33/article_3273.tsv
TSV_books/list_page_33/article_3283.tsv


TSV_books/list_page_59/article_5838.tsv
TSV_books/list_page_59/article_5859.tsv
TSV_books/list_page_59/article_5893.tsv
TSV_books/list_page_59/article_5897.tsv
TSV_books/list_page_59/article_5898.tsv
TSV_books/list_page_60/article_5901.tsv
TSV_books/list_page_60/article_5902.tsv
TSV_books/list_page_60/article_5951.tsv
TSV_books/list_page_60/article_5984.tsv
TSV_books/list_page_60/article_5986.tsv
TSV_books/list_page_60/article_5990.tsv
TSV_books/list_page_61/article_6002.tsv
TSV_books/list_page_61/article_6011.tsv
TSV_books/list_page_61/article_6012.tsv
TSV_books/list_page_61/article_6013.tsv
TSV_books/list_page_61/article_6062.tsv
TSV_books/list_page_61/article_6065.tsv
TSV_books/list_page_61/article_6097.tsv
TSV_books/list_page_61/article_6099.tsv
TSV_books/list_page_62/article_6100.tsv
TSV_books/list_page_62/article_6111.tsv
TSV_books/list_page_62/article_6112.tsv
TSV_books/list_page_62/article_6149.tsv
TSV_books/list_page_62/article_6170.tsv
TSV_books/list_page_62/article_6177.tsv


TSV_books/list_page_86/article_8528.tsv
TSV_books/list_page_86/article_8550.tsv
TSV_books/list_page_86/article_8561.tsv
TSV_books/list_page_86/article_8571.tsv
TSV_books/list_page_86/article_8579.tsv
TSV_books/list_page_86/article_8580.tsv
TSV_books/list_page_86/article_8595.tsv
TSV_books/list_page_87/article_8600.tsv
TSV_books/list_page_87/article_8602.tsv
TSV_books/list_page_87/article_8633.tsv
TSV_books/list_page_87/article_8634.tsv
TSV_books/list_page_87/article_8638.tsv
TSV_books/list_page_87/article_8658.tsv
TSV_books/list_page_87/article_8659.tsv
TSV_books/list_page_87/article_8672.tsv
TSV_books/list_page_87/article_8673.tsv
TSV_books/list_page_87/article_8675.tsv
TSV_books/list_page_87/article_8686.tsv
TSV_books/list_page_87/article_8689.tsv
TSV_books/list_page_87/article_8699.tsv
TSV_books/list_page_88/article_8716.tsv
TSV_books/list_page_88/article_8781.tsv
TSV_books/list_page_88/article_8783.tsv
TSV_books/list_page_89/article_8870.tsv
TSV_books/list_page_90/article_8905.tsv


TSV_books/list_page_116/article_11515.tsv
TSV_books/list_page_116/article_11521.tsv
TSV_books/list_page_116/article_11543.tsv
TSV_books/list_page_116/article_11576.tsv
TSV_books/list_page_116/article_11587.tsv
TSV_books/list_page_116/article_11590.tsv
TSV_books/list_page_116/article_11595.tsv
TSV_books/list_page_117/article_11609.tsv
TSV_books/list_page_117/article_11621.tsv
TSV_books/list_page_117/article_11625.tsv
TSV_books/list_page_117/article_11627.tsv
TSV_books/list_page_117/article_11638.tsv
TSV_books/list_page_117/article_11649.tsv
TSV_books/list_page_117/article_11651.tsv
TSV_books/list_page_117/article_11656.tsv
TSV_books/list_page_117/article_11657.tsv
TSV_books/list_page_117/article_11678.tsv
TSV_books/list_page_117/article_11687.tsv
TSV_books/list_page_118/article_11713.tsv
TSV_books/list_page_118/article_11720.tsv
TSV_books/list_page_118/article_11739.tsv
TSV_books/list_page_118/article_11743.tsv
TSV_books/list_page_118/article_11745.tsv
TSV_books/list_page_118/article_11

...

TSV_books/list_page_178/article_17734.tsv
TSV_books/list_page_178/article_17742.tsv
TSV_books/list_page_178/article_17778.tsv
TSV_books/list_page_178/article_17786.tsv
TSV_books/list_page_179/article_17808.tsv
TSV_books/list_page_179/article_17831.tsv
TSV_books/list_page_179/article_17842.tsv
TSV_books/list_page_179/article_17895.tsv
TSV_books/list_page_180/article_17925.tsv
TSV_books/list_page_180/article_17933.tsv
TSV_books/list_page_180/article_17937.tsv
TSV_books/list_page_180/article_17969.tsv
TSV_books/list_page_180/article_17992.tsv
TSV_books/list_page_181/article_18037.tsv
TSV_books/list_page_181/article_18046.tsv
TSV_books/list_page_181/article_18049.tsv
TSV_books/list_page_181/article_18089.tsv
TSV_books/list_page_181/article_18099.tsv
TSV_books/list_page_182/article_18106.tsv
TSV_books/list_page_182/article_18186.tsv
TSV_books/list_page_182/article_18198.tsv
TSV_books/list_page_183/article_18209.tsv
TSV_books/list_page_183/article_18219.tsv
TSV_books/list_page_183/article_18

TSV_books/list_page_206/article_20565.tsv
TSV_books/list_page_206/article_20574.tsv
TSV_books/list_page_206/article_20580.tsv
TSV_books/list_page_206/article_20584.tsv
TSV_books/list_page_206/article_20591.tsv
TSV_books/list_page_206/article_20599.tsv
TSV_books/list_page_207/article_20603.tsv
TSV_books/list_page_207/article_20620.tsv
TSV_books/list_page_207/article_20621.tsv
TSV_books/list_page_207/article_20627.tsv
TSV_books/list_page_207/article_20633.tsv
TSV_books/list_page_207/article_20653.tsv
TSV_books/list_page_207/article_20672.tsv
TSV_books/list_page_207/article_20679.tsv
TSV_books/list_page_207/article_20688.tsv
TSV_books/list_page_207/article_20697.tsv
TSV_books/list_page_208/article_20700.tsv
TSV_books/list_page_208/article_20701.tsv
TSV_books/list_page_208/article_20704.tsv
TSV_books/list_page_208/article_20709.tsv
TSV_books/list_page_208/article_20713.tsv
TSV_books/list_page_208/article_20714.tsv
TSV_books/list_page_208/article_20716.tsv
TSV_books/list_page_208/article_20

TSV_books/list_page_230/article_22975.tsv
TSV_books/list_page_230/article_22978.tsv
TSV_books/list_page_231/article_23008.tsv
TSV_books/list_page_231/article_23010.tsv
TSV_books/list_page_231/article_23019.tsv
TSV_books/list_page_231/article_23023.tsv
TSV_books/list_page_231/article_23033.tsv
TSV_books/list_page_231/article_23043.tsv
TSV_books/list_page_231/article_23060.tsv
TSV_books/list_page_231/article_23065.tsv
TSV_books/list_page_231/article_23084.tsv
TSV_books/list_page_232/article_23109.tsv
TSV_books/list_page_232/article_23123.tsv
TSV_books/list_page_232/article_23136.tsv
TSV_books/list_page_232/article_23140.tsv
TSV_books/list_page_232/article_23141.tsv
TSV_books/list_page_232/article_23150.tsv
TSV_books/list_page_232/article_23159.tsv
TSV_books/list_page_232/article_23171.tsv
TSV_books/list_page_232/article_23174.tsv
TSV_books/list_page_232/article_23182.tsv
TSV_books/list_page_232/article_23188.tsv
TSV_books/list_page_232/article_23192.tsv
TSV_books/list_page_232/article_23

TSV_books/list_page_256/article_25574.tsv
TSV_books/list_page_257/article_25611.tsv
TSV_books/list_page_257/article_25634.tsv
TSV_books/list_page_257/article_25635.tsv
TSV_books/list_page_257/article_25638.tsv
TSV_books/list_page_257/article_25642.tsv
TSV_books/list_page_257/article_25662.tsv
TSV_books/list_page_257/article_25668.tsv
TSV_books/list_page_257/article_25674.tsv
TSV_books/list_page_257/article_25683.tsv
TSV_books/list_page_257/article_25684.tsv
TSV_books/list_page_257/article_25692.tsv
TSV_books/list_page_258/article_25707.tsv
TSV_books/list_page_258/article_25718.tsv
TSV_books/list_page_258/article_25729.tsv
TSV_books/list_page_258/article_25735.tsv
TSV_books/list_page_258/article_25745.tsv
TSV_books/list_page_258/article_25759.tsv
TSV_books/list_page_258/article_25766.tsv
TSV_books/list_page_258/article_25769.tsv
TSV_books/list_page_258/article_25795.tsv
TSV_books/list_page_258/article_25797.tsv
TSV_books/list_page_259/article_25810.tsv
TSV_books/list_page_259/article_25

TSV_books/list_page_283/article_28295.tsv
TSV_books/list_page_284/article_28304.tsv
TSV_books/list_page_284/article_28324.tsv
TSV_books/list_page_284/article_28371.tsv
TSV_books/list_page_284/article_28394.tsv
TSV_books/list_page_285/article_28417.tsv
TSV_books/list_page_285/article_28427.tsv
TSV_books/list_page_285/article_28448.tsv
TSV_books/list_page_285/article_28467.tsv
TSV_books/list_page_285/article_28482.tsv
TSV_books/list_page_285/article_28494.tsv
TSV_books/list_page_286/article_28501.tsv
TSV_books/list_page_286/article_28502.tsv
TSV_books/list_page_286/article_28507.tsv
TSV_books/list_page_286/article_28521.tsv
TSV_books/list_page_286/article_28524.tsv
TSV_books/list_page_286/article_28527.tsv
TSV_books/list_page_286/article_28544.tsv
TSV_books/list_page_286/article_28558.tsv
TSV_books/list_page_286/article_28564.tsv
TSV_books/list_page_286/article_28567.tsv
TSV_books/list_page_286/article_28574.tsv
TSV_books/list_page_286/article_28597.tsv
TSV_books/list_page_287/article_28

#### 2.2.2 Execute the query

In [17]:
query = input('Query: ')
query = q2.preprocessor(query)
resultId = q2.search(query)
k = 5

with open('generated/documentNorm.json', 'r') as json_file:
    documentNorm = json.load(json_file)

with open('generated/invertedIndexTfidf.json', 'r') as json_file:
    invertedIndexTfidf = json.load(json_file)
    
with open('generated/vocabulary.json', 'r') as json_file:
    vocabulary = json.load(json_file)

heapResult = []
heapq.heapify(heapResult)
for result in resultId:
    norm = documentNorm[str(result)]

    summation = 0
    for token in query:
        summation += [x for x in invertedIndexTfidf[str(vocabulary[token])] if x[0] == int(result)][0][1]

    similarity = summation / norm

    heapq.heappush(heapResult, (similarity, result))

for i in heapq.nlargest(k, heapResult):
    with open(f'TSV_books/list_page_{i[1] // 100 + 1}/article_{i[1]}.tsv') as tsv_file:
        tsv_reader = csv.DictReader(tsv_file, delimiter='\t')
        row = tsv_reader.__next__()

    with open('generated/url_collected.txt') as urls:
        for j, url in enumerate(urls):
            if j == i[1]:
                print(f"Title: {row['bookTitle']}\nUrl: {url}\nSimilarity: {i[0]}\nPlot:\n{row['plot']}")
                print('--------------------------------------------------')

Query: survival games
Title: The Warden
Url: https://www.goodreads.com/book/show/32991547-natal-fault

Similarity: 0.44480051070972965
Plot:
Alice has led a normal life up until now. She wakes up finding herself trapped in a sick game of survival within the walls of an old asylum. She has to fight her way out while facing psychotic enemies, no one said they were all living psychopaths though... When she discovers how she got there she is forced to make difficult decisions. Will Alice survive the horror games?
--------------------------------------------------
Title: Devil's Own
Url: https://www.goodreads.com/book/show/1198929.Mountain_Solo

Similarity: 0.39352965240489546
Plot:
After surviving slavery, Aiden MacAlpin has nothing but thoughts of vengeance. When his tutor Elspeth learns a secret to his past, it thrusts them both into a game of passion and deception that neither may survive.
--------------------------------------------------
Title: U-10
Url: https://www.goodreads.com/book

# 3. Define a new score!

# 5. Algorithmic Question